In [1]:
import requests

import pandas as pd

from datetime import datetime

from pprint import pprint

In [6]:
lang = "es"
category = "demanda"
widget  = "evolucion"
query = "start_date=2023-01-01T00:00&end_date=2023-01-31T23:59&time_trunc=day"

In [7]:
headers = {
    "Accept": "application/json",
    "Content_Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [8]:
def demanda_datos(lang, category, widget, query):

    endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
    response = requests.get(url = endpoint, headers= headers)
    data = response.json()

    datetime_lista = list()
    value_lista = list()
    percentage_lista = list()


    response = requests.get(endpoint)
    data = response.json()

    for value in data['included']:
        for content in value['attributes']['values']:
                porcentaje = content['percentage']
                valor = content['value']
                fecha = content ['datetime']


                datetime_lista.append(fecha)
                value_lista.append(valor)
                percentage_lista.append(porcentaje)


    df = pd.DataFrame()
    df['datetime'] = datetime_lista
    df['percentage'] = percentage_lista
    df['value'] = value_lista

    df['datetime'] = df['datetime'].apply(lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.to_datetime(x, errors='coerce') is not pd.NaT else x)


    return df



In [9]:
demanda_datos(lang, category, widget, query)

,datetime,percentage,value
0,01/01/2023,1,501833.701
1,02/01/2023,1,617718.194
2,03/01/2023,1,672722.853
3,04/01/2023,1,684806.731
4,05/01/2023,1,667109.305
5,06/01/2023,1,575311.204
6,07/01/2023,1,620535.693
7,08/01/2023,1,608145.575
8,09/01/2023,1,697063.435
9,10/01/2023,1,726472.416


In [10]:
demanda_datos(lang, category, widget, query).to_csv('datos_demanda.csv')